In [1]:
#!/usr/bin/env python3
"""
Robot Fleet Management System - GUI Version
Maintenance Rotation & Tracking System with Tkinter UI
"""

import tkinter as tk
from tkinter import ttk, messagebox, filedialog, simpledialog
import json
import csv
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import random
import threading
import time

class Robot:
    """Represents a single robot in the fleet"""
    
    def __init__(self, robot_id: str, status: str, location: str):
        self.id = robot_id
        self.status = status  # 'active', 'spare', 'maintenance'
        self.location = location
        self.operating_hours = random.randint(500, 2500)
        self.last_maintenance = self._get_random_past_date(30)
        self.next_maintenance = self._get_random_future_date(30)
        self.maintenance_type = 'Scheduled'
        self.technician = ''
        self.notes = ''
        self.maintenance_log = []
    
    def _get_random_past_date(self, days_back: int) -> str:
        date = datetime.now() - timedelta(days=random.randint(1, days_back))
        return date.strftime('%Y-%m-%d')
    
    def _get_random_future_date(self, days_ahead: int) -> str:
        date = datetime.now() + timedelta(days=random.randint(1, days_ahead))
        return date.strftime('%Y-%m-%d')
    
    def days_until_maintenance(self) -> int:
        try:
            next_date = datetime.strptime(self.next_maintenance, '%Y-%m-%d')
            return (next_date - datetime.now()).days
        except:
            return 30
    
    def add_maintenance_log(self, description: str, technician: str = ''):
        entry = {
            'date': datetime.now().strftime('%Y-%m-%d %H:%M'),
            'description': description,
            'technician': technician or self.technician or 'System'
        }
        self.maintenance_log.insert(0, entry)
        self.maintenance_log = self.maintenance_log[:10]
    
    def to_dict(self) -> Dict:
        return {
            'id': self.id,
            'status': self.status,
            'location': self.location,
            'operating_hours': self.operating_hours,
            'last_maintenance': self.last_maintenance,
            'next_maintenance': self.next_maintenance,
            'maintenance_type': self.maintenance_type,
            'technician': self.technician,
            'notes': self.notes,
            'maintenance_log': self.maintenance_log
        }
    
    @classmethod
    def from_dict(cls, data: Dict) -> 'Robot':
        robot = cls(data['id'], data['status'], data['location'])
        robot.operating_hours = data.get('operating_hours', 0)
        robot.last_maintenance = data.get('last_maintenance', '')
        robot.next_maintenance = data.get('next_maintenance', '')
        robot.maintenance_type = data.get('maintenance_type', 'Scheduled')
        robot.technician = data.get('technician', '')
        robot.notes = data.get('notes', '')
        robot.maintenance_log = data.get('maintenance_log', [])
        return robot


class RobotFleetManager:
    """Main class for managing the robot fleet"""
    
    def __init__(self, data_file: str = 'robot_fleet_data.json'):
        self.data_file = data_file
        self.robots: Dict[str, Robot] = {}
        self.callbacks = []  # For UI updates
        self.load_data()
    
    def add_update_callback(self, callback):
        """Add callback for UI updates"""
        self.callbacks.append(callback)
    
    def notify_update(self):
        """Notify all callbacks of updates"""
        for callback in self.callbacks:
            try:
                callback()
            except:
                pass
    
    def initialize_fleet(self):
        active_robots = [
            ('EBA_R1', 'Empty Bag A Station'),
            ('EBA_R2', 'Empty Bag A Station'),
            ('EBA_R3', 'Empty Bag A Station'),
            ('EBB_R1', 'Empty Bag B Station'),
            ('EBB_R2', 'Empty Bag B Station'),
            ('EBB_R3', 'Empty Bag B Station'),
            ('FA_R1', 'Filler A Station'),
            ('FA_R2', 'Filler A Station'),
            ('FB_R1', 'Filler B Station'),
            ('FB_R2', 'Filler B Station'),
            ('PAL_RA', 'Palletizer Area'),
            ('PAL_RB', 'Palletizer Area')
        ]
        
        spare_robots = [
            ('SPARE_R1', 'Storage Bay'),
            ('SPARE_R2', 'Storage Bay'),
            ('SPARE_R3', 'Storage Bay'),
            ('SPARE_R4', 'Storage Bay')
        ]
        
        for robot_id, location in active_robots:
            robot = Robot(robot_id, 'active', location)
            self.robots[robot_id] = robot
        
        for robot_id, location in spare_robots:
            robot = Robot(robot_id, 'spare', location)
            robot.operating_hours = random.randint(100, 1000)
            self.robots[robot_id] = robot
        
        self.save_data()
    
    def load_data(self):
        if os.path.exists(self.data_file):
            try:
                with open(self.data_file, 'r') as f:
                    data = json.load(f)
                    for robot_id, robot_data in data.items():
                        self.robots[robot_id] = Robot.from_dict(robot_data)
            except Exception as e:
                self.initialize_fleet()
        else:
            self.initialize_fleet()
    
    def save_data(self):
        try:
            data = {robot_id: robot.to_dict() for robot_id, robot in self.robots.items()}
            with open(self.data_file, 'w') as f:
                json.dump(data, f, indent=2)
            self.notify_update()
            return True
        except Exception as e:
            return False
    
    def get_robots_by_status(self, status: str) -> List[Robot]:
        return [robot for robot in self.robots.values() if robot.status == status]
    
    def get_robot(self, robot_id: str) -> Optional[Robot]:
        return self.robots.get(robot_id)
    
    def get_fleet_stats(self):
        active_count = len(self.get_robots_by_status('active'))
        spare_count = len(self.get_robots_by_status('spare'))
        maintenance_count = len(self.get_robots_by_status('maintenance'))
        uptime = (active_count / (active_count + maintenance_count)) * 100 if (active_count + maintenance_count) > 0 else 100
        return {
            'active': active_count,
            'spare': spare_count,
            'maintenance': maintenance_count,
            'total': len(self.robots),
            'uptime': uptime
        }
    
    def perform_rotation(self, robot_out_id: str, robot_in_id: str, reason: str):
        robot_out = self.get_robot(robot_out_id)
        robot_in = self.get_robot(robot_in_id)
        
        if not robot_out or not robot_in:
            return False, "Robot not found"
        
        if robot_out.status != 'active':
            return False, f"{robot_out_id} is not currently active"
        
        if robot_in.status != 'spare':
            return False, f"{robot_in_id} is not currently spare"
        
        old_location = robot_out.location
        robot_out.status = 'spare'
        robot_out.location = 'Storage Bay'
        robot_in.status = 'active'
        robot_in.location = old_location
        
        robot_out.add_maintenance_log(f"Rotated out from {old_location} - Reason: {reason}")
        robot_in.add_maintenance_log(f"Rotated to {old_location} - Reason: {reason}")
        
        self.save_data()
        return True, f"Successfully rotated {robot_out_id} with {robot_in_id}"
    
    def start_maintenance(self, robot_id: str, maintenance_type: str, technician: str, notes: str = ''):
        robot = self.get_robot(robot_id)
        if not robot:
            return False, "Robot not found"
        
        if robot.status == 'maintenance':
            return False, "Robot is already in maintenance"
        
        robot.status = 'maintenance'
        robot.maintenance_type = maintenance_type
        robot.technician = technician
        robot.notes = notes
        
        robot.add_maintenance_log(f"Started {maintenance_type} maintenance", technician)
        self.save_data()
        return True, f"{robot_id} moved to maintenance"
    
    def complete_maintenance(self, robot_id: str, notes: str = ''):
        robot = self.get_robot(robot_id)
        if not robot:
            return False, "Robot not found"
        
        if robot.status != 'maintenance':
            return False, "Robot is not in maintenance"
        
        active_robots = ['EBA_R1', 'EBA_R2', 'EBA_R3', 'EBB_R1', 'EBB_R2', 'EBB_R3',
                        'FA_R1', 'FA_R2', 'FB_R1', 'FB_R2', 'PAL_RA', 'PAL_RB']
        
        if robot_id in active_robots:
            robot.status = 'active'
            if robot_id.startswith('EBA_'):
                robot.location = 'Empty Bag A Station'
            elif robot_id.startswith('EBB_'):
                robot.location = 'Empty Bag B Station'
            elif robot_id.startswith('FA_'):
                robot.location = 'Filler A Station'
            elif robot_id.startswith('FB_'):
                robot.location = 'Filler B Station'
            elif robot_id.startswith('PAL_'):
                robot.location = 'Palletizer Area'
        else:
            robot.status = 'spare'
            robot.location = 'Storage Bay'
        
        robot.last_maintenance = datetime.now().strftime('%Y-%m-%d')
        robot.next_maintenance = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
        robot.operating_hours = 0
        
        maintenance_notes = f"Completed {robot.maintenance_type} maintenance"
        if notes:
            maintenance_notes += f" - {notes}"
        
        robot.add_maintenance_log(maintenance_notes, robot.technician)
        robot.notes = notes
        
        self.save_data()
        return True, f"Maintenance completed for {robot_id}"
    
    def export_to_csv(self, filename: str):
        try:
            with open(filename, 'w', newline='') as csvfile:
                fieldnames = ['ID', 'Status', 'Location', 'Operating Hours', 
                             'Last Maintenance', 'Next Maintenance', 'Days Until Maintenance',
                             'Maintenance Type', 'Technician', 'Notes']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                
                writer.writeheader()
                for robot in self.robots.values():
                    writer.writerow({
                        'ID': robot.id,
                        'Status': robot.status,
                        'Location': robot.location,
                        'Operating Hours': robot.operating_hours,
                        'Last Maintenance': robot.last_maintenance,
                        'Next Maintenance': robot.next_maintenance,
                        'Days Until Maintenance': robot.days_until_maintenance(),
                        'Maintenance Type': robot.maintenance_type,
                        'Technician': robot.technician,
                        'Notes': robot.notes
                    })
            return True
        except Exception as e:
            return False


class RobotMaintenanceGUI:
    """Main GUI application class"""
    
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("🤖 Robot Fleet Management System")
        self.root.geometry("1400x900")
        self.root.configure(bg='#f0f0f0')
        
        # Initialize fleet manager
        self.fleet_manager = RobotFleetManager()
        self.fleet_manager.add_update_callback(self.refresh_all)
        
        # Create GUI elements
        self.setup_styles()
        self.create_widgets()
        self.refresh_all()
        
        # Auto-refresh every 30 seconds
        self.auto_refresh()
    
    def setup_styles(self):
        """Setup custom styles for the application"""
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Configure custom styles
        self.style.configure('Title.TLabel', font=('Arial', 16, 'bold'), background='#2c3e50', foreground='white')
        self.style.configure('Stats.TLabel', font=('Arial', 12, 'bold'), background='#3498db', foreground='white')
        self.style.configure('Header.TLabel', font=('Arial', 11, 'bold'), background='#ecf0f1')
        self.style.configure('Status.Active.TLabel', background='#d4edda', foreground='#155724')
        self.style.configure('Status.Spare.TLabel', background='#fff3cd', foreground='#856404')
        self.style.configure('Status.Maintenance.TLabel', background='#f8d7da', foreground='#721c24')
    
    def create_widgets(self):
        """Create the main GUI widgets"""
        # Main container
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Title
        title_frame = ttk.Frame(main_frame)
        title_frame.pack(fill=tk.X, pady=(0, 10))
        
        title_label = ttk.Label(title_frame, text="🤖 Robot Fleet Management System", 
                               style='Title.TLabel')
        title_label.pack(pady=10)
        
        # Stats section
        self.create_stats_section(main_frame)
        
        # Main content area with notebook
        self.notebook = ttk.Notebook(main_frame)
        self.notebook.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Create tabs
        self.create_fleet_overview_tab()
        self.create_rotation_tab()
        self.create_maintenance_tab()
        self.create_details_tab()
        
        # Bottom buttons
        self.create_bottom_buttons(main_frame)
    
    def create_stats_section(self, parent):
        """Create the statistics section"""
        stats_frame = ttk.LabelFrame(parent, text="Fleet Statistics", padding=10)
        stats_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Stats grid
        stats_grid = ttk.Frame(stats_frame)
        stats_grid.pack(fill=tk.X)
        
        # Configure grid columns
        for i in range(5):
            stats_grid.columnconfigure(i, weight=1)
        
        # Stats labels
        self.stats_labels = {}
        stats_info = [
            ('active', 'Active Robots', '#27ae60'),
            ('spare', 'Spare Robots', '#f39c12'),
            ('maintenance', 'In Maintenance', '#e74c3c'),
            ('total', 'Total Fleet', '#3498db'),
            ('uptime', 'Fleet Uptime', '#9b59b6')
        ]
        
        for i, (key, label, color) in enumerate(stats_info):
            frame = tk.Frame(stats_grid, bg=color, relief=tk.RAISED, bd=2)
            frame.grid(row=0, column=i, padx=5, pady=5, sticky='ew')
            
            value_label = tk.Label(frame, text="0", font=('Arial', 24, 'bold'), 
                                 bg=color, fg='white')
            value_label.pack(pady=(10, 5))
            
            text_label = tk.Label(frame, text=label, font=('Arial', 10), 
                                bg=color, fg='white')
            text_label.pack(pady=(0, 10))
            
            self.stats_labels[key] = value_label
    
    def create_fleet_overview_tab(self):
        """Create the fleet overview tab"""
        tab_frame = ttk.Frame(self.notebook)
        self.notebook.add(tab_frame, text="Fleet Overview")
        
        # Filter section
        filter_frame = ttk.LabelFrame(tab_frame, text="Filters", padding=10)
        filter_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Label(filter_frame, text="Status Filter:").pack(side=tk.LEFT, padx=(0, 10))
        
        self.status_filter = ttk.Combobox(filter_frame, values=['All', 'Active', 'Spare', 'Maintenance'],
                                         state='readonly', width=15)
        self.status_filter.set('All')
        self.status_filter.pack(side=tk.LEFT, padx=(0, 10))
        self.status_filter.bind('<<ComboboxSelected>>', self.filter_robots)
        
        refresh_btn = ttk.Button(filter_frame, text="🔄 Refresh", command=self.refresh_robot_list)
        refresh_btn.pack(side=tk.LEFT, padx=(10, 0))
        
        # Robot list
        list_frame = ttk.Frame(tab_frame)
        list_frame.pack(fill=tk.BOTH, expand=True)
        
        # Treeview for robot list
        columns = ('ID', 'Status', 'Location', 'Hours', 'Next Maintenance', 'Days Left', 'Technician')
        self.robot_tree = ttk.Treeview(list_frame, columns=columns, show='headings', height=20)
        
        # Configure columns
        for col in columns:
            self.robot_tree.heading(col, text=col)
            self.robot_tree.column(col, width=120)
        
        # Scrollbars
        v_scrollbar = ttk.Scrollbar(list_frame, orient=tk.VERTICAL, command=self.robot_tree.yview)
        h_scrollbar = ttk.Scrollbar(list_frame, orient=tk.HORIZONTAL, command=self.robot_tree.xview)
        self.robot_tree.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)
        
        # Pack treeview and scrollbars
        self.robot_tree.grid(row=0, column=0, sticky='nsew')
        v_scrollbar.grid(row=0, column=1, sticky='ns')
        h_scrollbar.grid(row=1, column=0, sticky='ew')
        
        list_frame.grid_rowconfigure(0, weight=1)
        list_frame.grid_columnconfigure(0, weight=1)
        
        # Bind double-click to show details
        self.robot_tree.bind('<Double-1>', self.on_robot_double_click)
    
    def create_rotation_tab(self):
        """Create the robot rotation tab"""
        tab_frame = ttk.Frame(self.notebook)
        self.notebook.add(tab_frame, text="Robot Rotation")
        
        # Instructions
        instructions = ttk.Label(tab_frame, 
                               text="Select an active robot and a spare robot to perform rotation",
                               font=('Arial', 12))
        instructions.pack(pady=10)
        
        # Rotation form
        form_frame = ttk.LabelFrame(tab_frame, text="Rotation Setup", padding=20)
        form_frame.pack(fill=tk.X, padx=20, pady=10)
        
        # Robot out selection
        ttk.Label(form_frame, text="Robot to Rotate Out:").grid(row=0, column=0, sticky='w', pady=5)
        self.robot_out_var = tk.StringVar()
        self.robot_out_combo = ttk.Combobox(form_frame, textvariable=self.robot_out_var, 
                                           state='readonly', width=30)
        self.robot_out_combo.grid(row=0, column=1, padx=(10, 0), pady=5)
        
        # Robot in selection
        ttk.Label(form_frame, text="Replacement Robot:").grid(row=1, column=0, sticky='w', pady=5)
        self.robot_in_var = tk.StringVar()
        self.robot_in_combo = ttk.Combobox(form_frame, textvariable=self.robot_in_var, 
                                          state='readonly', width=30)
        self.robot_in_combo.grid(row=1, column=1, padx=(10, 0), pady=5)
        
        # Reason selection
        ttk.Label(form_frame, text="Rotation Reason:").grid(row=2, column=0, sticky='w', pady=5)
        self.rotation_reason_var = tk.StringVar(value='Scheduled Maintenance')
        reason_combo = ttk.Combobox(form_frame, textvariable=self.rotation_reason_var,
                                   values=['Scheduled Maintenance', 'Operating Hours Limit', 
                                          'Performance Issue', 'Preventive Care'], 
                                   state='readonly', width=30)
        reason_combo.grid(row=2, column=1, padx=(10, 0), pady=5)
        
        # Execute button
        execute_btn = ttk.Button(form_frame, text="🔄 Execute Rotation", 
                                command=self.execute_rotation, style='Accent.TButton')
        execute_btn.grid(row=3, column=0, columnspan=2, pady=20)
        
        # Configure grid weights
        form_frame.columnconfigure(1, weight=1)
    
    def create_maintenance_tab(self):
        """Create the maintenance management tab"""
        tab_frame = ttk.Frame(self.notebook)
        self.notebook.add(tab_frame, text="Maintenance")
        
        # Create paned window for maintenance operations
        paned = ttk.PanedWindow(tab_frame, orient=tk.HORIZONTAL)
        paned.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Start maintenance frame
        start_frame = ttk.LabelFrame(paned, text="Start Maintenance", padding=15)
        paned.add(start_frame, weight=1)
        
        ttk.Label(start_frame, text="Robot ID:").grid(row=0, column=0, sticky='w', pady=5)
        self.maint_robot_var = tk.StringVar()
        self.maint_robot_combo = ttk.Combobox(start_frame, textvariable=self.maint_robot_var,
                                             state='readonly', width=25)
        self.maint_robot_combo.grid(row=0, column=1, padx=(10, 0), pady=5)
        
        ttk.Label(start_frame, text="Maintenance Type:").grid(row=1, column=0, sticky='w', pady=5)
        self.maint_type_var = tk.StringVar(value='Scheduled')
        maint_type_combo = ttk.Combobox(start_frame, textvariable=self.maint_type_var,
                                       values=['Scheduled', 'Preventive', 'Corrective', 'Emergency'],
                                       state='readonly', width=25)
        maint_type_combo.grid(row=1, column=1, padx=(10, 0), pady=5)
        
        ttk.Label(start_frame, text="Technician:").grid(row=2, column=0, sticky='w', pady=5)
        self.technician_var = tk.StringVar()
        technician_entry = ttk.Entry(start_frame, textvariable=self.technician_var, width=25)
        technician_entry.grid(row=2, column=1, padx=(10, 0), pady=5)
        
        ttk.Label(start_frame, text="Notes:").grid(row=3, column=0, sticky='w', pady=5)
        self.maint_notes_text = tk.Text(start_frame, height=4, width=25)
        self.maint_notes_text.grid(row=3, column=1, padx=(10, 0), pady=5)
        
        start_btn = ttk.Button(start_frame, text="🔧 Start Maintenance",
                              command=self.start_maintenance)
        start_btn.grid(row=4, column=0, columnspan=2, pady=15)
        
        # Complete maintenance frame
        complete_frame = ttk.LabelFrame(paned, text="Complete Maintenance", padding=15)
        paned.add(complete_frame, weight=1)
        
        ttk.Label(complete_frame, text="Robot in Maintenance:").grid(row=0, column=0, sticky='w', pady=5)
        self.complete_robot_var = tk.StringVar()
        self.complete_robot_combo = ttk.Combobox(complete_frame, textvariable=self.complete_robot_var,
                                                state='readonly', width=25)
        self.complete_robot_combo.grid(row=0, column=1, padx=(10, 0), pady=5)
        
        ttk.Label(complete_frame, text="Completion Notes:").grid(row=1, column=0, sticky='w', pady=5)
        self.complete_notes_text = tk.Text(complete_frame, height=6, width=25)
        self.complete_notes_text.grid(row=1, column=1, padx=(10, 0), pady=5)
        
        complete_btn = ttk.Button(complete_frame, text="✅ Complete Maintenance",
                                 command=self.complete_maintenance)
        complete_btn.grid(row=2, column=0, columnspan=2, pady=15)
        
        # Configure grid weights
        start_frame.columnconfigure(1, weight=1)
        complete_frame.columnconfigure(1, weight=1)
    
    def create_details_tab(self):
        """Create the robot details tab"""
        tab_frame = ttk.Frame(self.notebook)
        self.notebook.add(tab_frame, text="Robot Details")
        
        # Robot selection
        select_frame = ttk.Frame(tab_frame)
        select_frame.pack(fill=tk.X, padx=10, pady=10)
        
        ttk.Label(select_frame, text="Select Robot:").pack(side=tk.LEFT, padx=(0, 10))
        self.detail_robot_var = tk.StringVar()
        self.detail_robot_combo = ttk.Combobox(select_frame, textvariable=self.detail_robot_var,
                                              state='readonly', width=20)
        self.detail_robot_combo.pack(side=tk.LEFT, padx=(0, 10))
        self.detail_robot_combo.bind('<<ComboboxSelected>>', self.show_robot_details)
        
        # Details display
        details_frame = ttk.LabelFrame(tab_frame, text="Robot Information", padding=15)
        details_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Create text widget for details
        self.details_text = tk.Text(details_frame, height=20, width=80, font=('Courier', 10))
        details_scrollbar = ttk.Scrollbar(details_frame, orient=tk.VERTICAL, command=self.details_text.yview)
        self.details_text.configure(yscrollcommand=details_scrollbar.set)
        
        self.details_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        details_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
    
    def create_bottom_buttons(self, parent):
        """Create bottom action buttons"""
        button_frame = ttk.Frame(parent)
        button_frame.pack(fill=tk.X, pady=10)
        
        # Left side buttons
        left_buttons = ttk.Frame(button_frame)
        left_buttons.pack(side=tk.LEFT)
        
        refresh_btn = ttk.Button(left_buttons, text="🔄 Refresh All", command=self.refresh_all)
        refresh_btn.pack(side=tk.LEFT, padx=(0, 10))
        
        export_btn = ttk.Button(left_buttons, text="📊 Export CSV", command=self.export_data)
        export_btn.pack(side=tk.LEFT, padx=(0, 10))
        
        # Right side buttons
        right_buttons = ttk.Frame(button_frame)
        right_buttons.pack(side=tk.RIGHT)
        
        save_btn = ttk.Button(right_buttons, text="💾 Save Data", command=self.save_data)
        save_btn.pack(side=tk.LEFT, padx=(0, 10))
        
        exit_btn = ttk.Button(right_buttons, text="❌ Exit", command=self.on_closing)
        exit_btn.pack(side=tk.LEFT)
    
    def refresh_all(self):
        """Refresh all data and UI elements"""
        self.update_stats()
        self.refresh_robot_list()
        self.update_combo_boxes()
    
    def update_stats(self):
        """Update the statistics display"""
        stats = self.fleet_manager.get_fleet_stats()
        
        self.stats_labels['active'].config(text=str(stats['active']))
        self.stats_labels['spare'].config(text=str(stats['spare']))
        self.stats_labels['maintenance'].config(text=str(stats['maintenance']))
        self.stats_labels['total'].config(text=str(stats['total']))
        self.stats_labels['uptime'].config(text=f"{stats['uptime']:.1f}%")
    
    def refresh_robot_list(self):
        """Refresh the robot list in the overview tab"""
        # Clear existing items
        for item in self.robot_tree.get_children():
            self.robot_tree.delete(item)
        
        # Get filter value
        filter_status = self.status_filter.get().lower()
        
        # Add robots to tree
        robots = list(self.fleet_manager.robots.values())
        robots.sort(key=lambda r: (r.status, r.id))
        
        for robot in robots:
            if filter_status == 'all' or robot.status == filter_status:
                days_left = robot.days_until_maintenance()
                
                # Color coding based on urgency
                if days_left < 7:
                    tags = ('urgent',)
                elif days_left < 14:
                    tags = ('warning',)
                else:
                    tags = ('normal',)
                
                self.robot_tree.insert('', 'end', values=(
                    robot.id,
                    robot.status.title(),
                    robot.location,
                    robot.operating_hours,
                    robot.next_maintenance,
                    f"{days_left} days",
                    robot.technician or 'Not assigned'
                ), tags=tags)
        
        # Configure tags for color coding
        self.robot_tree.tag_configure('urgent', background='#ffebee')
        self.robot_tree.tag_configure('warning', background='#fff3e0')
        self.robot_tree.tag_configure('normal', background='#e8f5e8')
    
    def update_combo_boxes(self):
        """Update all combo box values"""
        # Get robots by status
        active_robots = [r.id for r in self.fleet_manager.get_robots_by_status('active')]
        spare_robots = [r.id for r in self.fleet_manager.get_robots_by_status('spare')]
        maintenance_robots = [r.id for r in self.fleet_manager.get_robots_by_status('maintenance')]
        all_robots = list(self.fleet_manager.robots.keys())
        
        # Update rotation combos
        self.robot_out_combo['values'] = active_robots
        self.robot_in_combo['values'] = spare_robots
        
        # Update maintenance combos
        available_for_maintenance = active_robots + spare_robots
        self.maint_robot_combo['values'] = available_for_maintenance
        self.complete_robot_combo['values'] = maintenance_robots
        
        # Update details combo
        self.detail_robot_combo['values'] = sorted(all_robots)
        
        # Clear selections if no longer valid
        if self.robot_out_var.get() not in active_robots:
            self.robot_out_var.set('')
        if self.robot_in_var.get() not in spare_robots:
            self.robot_in_var.set('')
        if self.maint_robot_var.get() not in available_for_maintenance:
            self.maint_robot_var.set('')
        if self.complete_robot_var.get() not in maintenance_robots:
            self.complete_robot_var.set('')
    
    def filter_robots(self, event=None):
        """Filter robots based on status selection"""
        self.refresh_robot_list()
    
    def on_robot_double_click(self, event):
        """Handle double-click on robot in tree view"""
        selection = self.robot_tree.selection()
        if selection:
            item = self.robot_tree.item(selection[0])
            robot_id = item['values'][0]
            
            # Switch to details tab and show robot
            self.notebook.select(3)  # Details tab index
            self.detail_robot_var.set(robot_id)
            self.show_robot_details()
    
    def execute_rotation(self):
        """Execute robot rotation"""
        robot_out = self.robot_out_var.get()
        robot_in = self.robot_in_var.get()
        reason = self.rotation_reason_var.get()
        
        if not robot_out or not robot_in:
            messagebox.showerror("Error", "Please select both robots for rotation")
            return
        
        success, message = self.fleet_manager.perform_rotation(robot_out, robot_in, reason)
        
        if success:
            messagebox.showinfo("Success", message)
            self.robot_out_var.set('')
            self.robot_in_var.set('')
            self.refresh_all()
        else:
            messagebox.showerror("Error", message)
    
    def start_maintenance(self):
        """Start maintenance on selected robot"""
        robot_id = self.maint_robot_var.get()
        maintenance_type = self.maint_type_var.get()
        technician = self.technician_var.get()
        notes = self.maint_notes_text.get('1.0', tk.END).strip()
        
        if not robot_id:
            messagebox.showerror("Error", "Please select a robot")
            return
        
        if not technician:
            messagebox.showerror("Error", "Please enter technician name")
            return
        
        success, message = self.fleet_manager.start_maintenance(robot_id, maintenance_type, technician, notes)
        
        if success:
            messagebox.showinfo("Success", message)
            self.maint_robot_var.set('')
            self.technician_var.set('')
            self.maint_notes_text.delete('1.0', tk.END)
            self.refresh_all()
        else:
            messagebox.showerror("Error", message)
    
    def complete_maintenance(self):
        """Complete maintenance on selected robot"""
        robot_id = self.complete_robot_var.get()
        notes = self.complete_notes_text.get('1.0', tk.END).strip()
        
        if not robot_id:
            messagebox.showerror("Error", "Please select a robot")
            return
        
        success, message = self.fleet_manager.complete_maintenance(robot_id, notes)
        
        if success:
            messagebox.showinfo("Success", message)
            self.complete_robot_var.set('')
            self.complete_notes_text.delete('1.0', tk.END)
            self.refresh_all()
        else:
            messagebox.showerror("Error", message)
    
    def show_robot_details(self, event=None):
        """Show detailed information for selected robot"""
        robot_id = self.detail_robot_var.get()
        if not robot_id:
            return
        
        robot = self.fleet_manager.get_robot(robot_id)
        if not robot:
            return
        
        # Clear previous details
        self.details_text.delete('1.0', tk.END)
        
        # Format robot details
        days_left = robot.days_until_maintenance()
        urgency = "🔴 URGENT" if days_left < 7 else "🟡 SOON" if days_left < 14 else "🟢 NORMAL"
        
        details = f"""
🤖 ROBOT DETAILS: {robot.id}
{'='*60}

📊 Basic Information:
   Status:             {robot.status.upper()}
   Location:           {robot.location}
   Operating Hours:    {robot.operating_hours:,}
   
🔧 Maintenance Information:
   Last Maintenance:   {robot.last_maintenance}
   Next Maintenance:   {robot.next_maintenance}
   Days Until:         {days_left} days ({urgency})
   Maintenance Type:   {robot.maintenance_type}
   Assigned Tech:      {robot.technician or 'Not assigned'}
   
📝 Notes:
   {robot.notes or 'No notes available'}

🔍 Maintenance History:
{'='*60}
"""
        
        if robot.maintenance_log:
            for i, entry in enumerate(robot.maintenance_log[:10], 1):
                details += f"\n{i:2d}. {entry['date']} - {entry['description']}"
                details += f"\n    Technician: {entry['technician']}\n"
        else:
            details += "\nNo maintenance history available.\n"
        
        # Insert details into text widget
        self.details_text.insert('1.0', details)
        
        # Configure text colors for better readability
        self.details_text.tag_add("header", "1.0", "3.0")
        self.details_text.tag_config("header", font=('Courier', 12, 'bold'), foreground='#2c3e50')
    
    def export_data(self):
        """Export robot data to CSV"""
        filename = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("All files", "*.*")],
            title="Export Robot Data"
        )
        
        if filename:
            success = self.fleet_manager.export_to_csv(filename)
            if success:
                messagebox.showinfo("Success", f"Data exported successfully to {filename}")
            else:
                messagebox.showerror("Error", "Failed to export data")
    
    def save_data(self):
        """Save current data"""
        success = self.fleet_manager.save_data()
        if success:
            messagebox.showinfo("Success", "Data saved successfully")
        else:
            messagebox.showerror("Error", "Failed to save data")
    
    def auto_refresh(self):
        """Auto-refresh the UI every 30 seconds"""
        self.refresh_all()
        # Schedule next refresh
        self.root.after(30000, self.auto_refresh)
    
    def on_closing(self):
        """Handle application closing"""
        if messagebox.askokcancel("Quit", "Do you want to save data before quitting?"):
            self.fleet_manager.save_data()
        self.root.quit()
        self.root.destroy()
    
    def run(self):
        """Start the GUI application"""
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.root.mainloop()


class SplashScreen:
    """Splash screen for application startup"""
    
    def __init__(self):
        self.splash = tk.Toplevel()
        self.splash.title("Loading...")
        self.splash.geometry("400x300")
        self.splash.configure(bg='#2c3e50')
        self.splash.resizable(False, False)
        
        # Center the splash screen
        self.splash.transient()
        self.splash.grab_set()
        
        # Create splash content
        self.create_splash_content()
        
        # Show splash for 3 seconds
        self.splash.after(3000, self.close_splash)
    
    def create_splash_content(self):
        """Create splash screen content"""
        # Title
        title_label = tk.Label(self.splash, text="🤖 Robot Fleet Management",
                              font=('Arial', 20, 'bold'), bg='#2c3e50', fg='white')
        title_label.pack(pady=50)
        
        # Subtitle
        subtitle_label = tk.Label(self.splash, text="Maintenance & Rotation System",
                                 font=('Arial', 14), bg='#2c3e50', fg='#ecf0f1')
        subtitle_label.pack(pady=10)
        
        # Loading animation
        self.loading_label = tk.Label(self.splash, text="Loading...",
                                     font=('Arial', 12), bg='#2c3e50', fg='#3498db')
        self.loading_label.pack(pady=30)
        
        # Progress simulation
        self.animate_loading()
    
    def animate_loading(self):
        """Animate loading text"""
        current = self.loading_label.cget("text")
        if current == "Loading...":
            self.loading_label.config(text="Loading")
        elif current == "Loading":
            self.loading_label.config(text="Loading.")
        elif current == "Loading.":
            self.loading_label.config(text="Loading..")
        else:
            self.loading_label.config(text="Loading...")
        
        if hasattr(self, 'splash') and self.splash.winfo_exists():
            self.splash.after(200, self.animate_loading)
    
    def close_splash(self):
        """Close splash screen"""
        if hasattr(self, 'splash'):
            self.splash.destroy()


def main():
    """Main function to start the application"""
    try:
        # Show splash screen
        root = tk.Tk()
        root.withdraw()  # Hide main window initially
        
        splash = SplashScreen()
        root.wait_window(splash.splash)
        
        # Show main application
        root.deiconify()
        root.withdraw()  # Hide again, let GUI class handle it
        
        # Start main application
        app = RobotMaintenanceGUI()
        app.run()
        
    except Exception as e:
        messagebox.showerror("Error", f"Failed to start application: {str(e)}")


if __name__ == "__main__":
    main()